In [12]:
import pandas as pd

from backtesting import Strategy, Backtest
from backtesting.lib import crossover

import talib
import pandas_ta

In [38]:
sbi = pd.read_parquet("../res/sbin.parquet")
sbi

,Open,High,Low,Close,Volume
ts,,,,,
2024-01-05 03:45:00,645.00,647.45,645.00,646.25,425786
2024-01-05 03:48:00,646.20,647.75,646.20,646.80,243890
2024-01-05 03:51:00,646.90,650.00,646.80,649.20,609582
2024-01-05 03:54:00,649.60,649.60,648.00,648.45,232613
2024-01-05 03:57:00,648.45,649.00,648.00,648.50,152385
...,...,...,...,...,...
2024-01-23 09:45:00,605.45,605.55,604.25,604.65,481332
2024-01-23 09:48:00,604.70,605.90,604.30,605.05,561024
2024-01-23 09:51:00,605.05,606.65,605.00,606.05,480432


In [39]:
class EMACross(Strategy):
    fast_window = 13
    slow_window = 26

    def init(self):
        close = self.data.Close
        self.ema_fast = self.I(talib.EMA, close, self.fast_window)
        self.ema_slow = self.I(talib.EMA, close, self.slow_window)

    def next(self):
        if crossover(self.ema_fast, self.ema_slow):
            self.buy()
        elif crossover(self.ema_slow, self.ema_fast):
            self.sell()

In [40]:
class VWAPCross(Strategy):
    def init(self):
        df = self.data.df
        self.vwap = self.I(pandas_ta.overlap.vwap, df.High, df.Low, df.Close, df.Volume)

    def next(self):
        if crossover(self.data.Close, self.vwap):
            self.buy()
        elif crossover(self.vwap, self.data.Close):
            self.sell()

In [41]:
bt = Backtest(sbi, VWAPCross, cash=100_000, exclusive_orders=True)
bt.run()

Start                     2024-01-05 03:45:00
End                       2024-01-23 09:57:00
Duration                     18 days 06:12:00
Exposure Time [%]                   99.630769
Equity Final [$]                    105565.05
Equity Peak [$]                     106293.05
Return [%]                            5.56505
Buy & Hold Return [%]               -6.375242
Return (Ann.) [%]                   156.93589
Volatility (Ann.) [%]               68.560355
Sharpe Ratio                         2.289018
Sortino Ratio                       15.196095
Calmar Ratio                        52.961725
Max. Drawdown [%]                   -2.963194
Avg. Drawdown [%]                   -0.502905
Max. Drawdown Duration        8 days 00:51:00
Avg. Drawdown Duration        0 days 17:28:00
# Trades                                  117
Win Rate [%]                        22.222222
Best Trade [%]                       4.152167
Worst Trade [%]                     -0.691152
Avg. Trade [%]                    

In [42]:
bt.plot()

/home/cognusboi/workspace/programming/python/algotrading/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/cognusboi/workspace/programming/python/algotrading/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/cognusboi/workspace/programming/python/algotrading/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(resample_rule, label='left')
/home/cognusboi/workspace/programming/python/algotrading/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag

GridPlot(id='p1860', ...)